In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import copy


import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset,random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models

In [2]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
import os

In [4]:
df=pd.read_csv("./new_train.csv",)
print(df.head())
df.shape

     Image         target
0   96.jpg       manipuri
1  163.jpg  bharatanatyam
2  450.jpg         odissi
3  219.jpg      kathakali
4  455.jpg         odissi


(46592, 2)

In [5]:
train={}
for image,label in zip(df["Image"],df["target"]):
  train[image]=label

In [6]:
class DancePredictionDataset(Dataset):

  def __init__(self,file_path,train=True,transform=None):
    self.transform=transform
    self.file_path=file_path
    self.train=train
    self.file_names=[file for _,_,files in os.walk(self.file_path) for file in files]
    self.len=len(self.file_names)
    if self.train:
      self.classes_mapping=self.get_classes()

  def __len__(self):
    return len(self.file_names)

    
  def pil_loader(self,path):
    with open(path,"rb") as f:
      img=Image.open(f)
      return img.convert("RGB")

  def __getitem__(self,index):
    file_name=self.file_names[index]
    image_data=self.pil_loader(self.file_path+"/"+file_name)
    if self.transform:
      image_data=self.transform(image_data)

    if self.train:
      class_=train[file_name]
      label=self.classes_mapping[class_]
      return image_data,label
    else:
      return image_data,file_name

  def get_classes(self):
    class_mapping={}
    class_mapping["manipuri"]=torch.tensor([1,0,0,0,0,0,0,0])
    class_mapping["bharatanatyam"]=torch.tensor([0,1,0,0,0,0,0,0])
    class_mapping["odissi"]=torch.tensor([0,0,1,0,0,0,0,0])
    class_mapping["kathakali"]=torch.tensor([0,0,0,1,0,0,0,0])
    class_mapping["kathak"]=torch.tensor([0,0,0,0,1,0,0,0])
    class_mapping["sattriya"]=torch.tensor([0,0,0,0,0,1,0,0])
    class_mapping["kuchipudi"]=torch.tensor([0,0,0,0,0,0,1,0])
    class_mapping["mohiniyattam"]=torch.tensor([0,0,0,0,0,0,0,1])
    return class_mapping

In [7]:
transform=transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),transforms.ToTensor()])

In [8]:
full_data = DancePredictionDataset("./dataset/aug_train/",train=True,transform=transform)
train_size = int(0.95 * len(full_data))
test_size = len(full_data) - train_size

train_data, validation_data = random_split(full_data, [train_size, test_size])
batch_size=128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=True)

In [9]:
mapping={}
mapping[1]="manipuri"
mapping[2]="bharatanatyam"
mapping[3]="odissi"
mapping[4]="kathakali"
mapping[5]="kathak"
mapping[6]="sattriya"
mapping[7]="kuchipudi"
mapping[8]="mohiniyattam"

In [10]:
class Net(nn.Module):
    
    
    def __init__(self):
        super(Net,self).__init__()
        self.feature=resnet         
        
    def forward(self,input_):
        input_=input_.to(device)
        out=self.feature(input_)        
        return out

In [11]:
def eval(net,dataloader,device="cpu"):
    total=0
    correct=0
    for data in dataloader:
        net.eval()
        img,label=data
        img,label=img.to(device),label.to(device)
        pred=net(img)
        for i in range(pred.shape[0]):
            total+=1
            if (pred[i].argmax()==label[i].argmax()):
                correct+=1
                
        del img,label,pred
        torch.cuda.empty_cache
    
    accuracy=(correct/total)*100
    return accuracy

In [12]:
resnet=models.resnet18(pretrained=True)
net=Net()

for param in net.parameters():
    param.requires_grad=False
in_features=net.feature.fc.in_features
net.feature.fc=nn.Sequential(
    nn.Linear(512,256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,16),
    nn.ReLU(),
    nn.Linear(16,8),
)

In [13]:
net.to(device)
loss_fn=nn.CrossEntropyLoss()
opt=optim.Adam(net.parameters(),lr=0.0005,weight_decay=0.001)

In [22]:
epochs=2 #2+8+1+2 total epochs  #  last two were over-fitting the data
net.train()
for e in range(epochs):
    net.train()
    avg_loss_per_epoch=0
    for i,data in enumerate(train_loader,0):
        opt.zero_grad()
        input_,output=data
        input_,output=input_.to(device),output.to(device)
        pred=net(input_) 
        output=output.argmax(-1)
        loss=loss_fn(pred,output)
        loss.backward()
        opt.step()
        avg_loss_per_epoch+=loss.item()
        print("loss at epoch : {} and iter : {} is : {} ".format(e+1,i+1,loss.item()))
        del input_,output,pred
        torch.cuda.empty_cache()
    print("loss at avg_loss_epoch : {} is : {}".format(e+1,avg_loss_per_epoch/(train_size/batch_size)))
    print(eval(net,validation_loader,device=device))

loss at epoch : 1 and iter : 1 is : 0.059484805911779404 
loss at epoch : 1 and iter : 2 is : 0.06191911920905113 
loss at epoch : 1 and iter : 3 is : 0.1330948770046234 
loss at epoch : 1 and iter : 4 is : 0.06609419733285904 
loss at epoch : 1 and iter : 5 is : 0.14373166859149933 
loss at epoch : 1 and iter : 6 is : 0.05597575381398201 
loss at epoch : 1 and iter : 7 is : 0.08578996360301971 
loss at epoch : 1 and iter : 8 is : 0.06201861798763275 
loss at epoch : 1 and iter : 9 is : 0.14132125675678253 
loss at epoch : 1 and iter : 10 is : 0.0601663812994957 
loss at epoch : 1 and iter : 11 is : 0.1108313649892807 
loss at epoch : 1 and iter : 12 is : 0.04910026490688324 
loss at epoch : 1 and iter : 13 is : 0.10065719485282898 
loss at epoch : 1 and iter : 14 is : 0.08561535179615021 
loss at epoch : 1 and iter : 15 is : 0.044427238404750824 
loss at epoch : 1 and iter : 16 is : 0.09578797221183777 
loss at epoch : 1 and iter : 17 is : 0.070526622235775 
loss at epoch : 1 and iter

loss at epoch : 1 and iter : 142 is : 0.07729572802782059 
loss at epoch : 1 and iter : 143 is : 0.06276071071624756 
loss at epoch : 1 and iter : 144 is : 0.02542017214000225 
loss at epoch : 1 and iter : 145 is : 0.05816001445055008 
loss at epoch : 1 and iter : 146 is : 0.054328497499227524 
loss at epoch : 1 and iter : 147 is : 0.06179991737008095 
loss at epoch : 1 and iter : 148 is : 0.07741239666938782 
loss at epoch : 1 and iter : 149 is : 0.06781169772148132 
loss at epoch : 1 and iter : 150 is : 0.05262553691864014 
loss at epoch : 1 and iter : 151 is : 0.08597727864980698 
loss at epoch : 1 and iter : 152 is : 0.13863949477672577 
loss at epoch : 1 and iter : 153 is : 0.06604313850402832 
loss at epoch : 1 and iter : 154 is : 0.08679341524839401 
loss at epoch : 1 and iter : 155 is : 0.10362443327903748 
loss at epoch : 1 and iter : 156 is : 0.07621819525957108 
loss at epoch : 1 and iter : 157 is : 0.07075319439172745 
loss at epoch : 1 and iter : 158 is : 0.068522065877914

loss at epoch : 1 and iter : 281 is : 0.1018671914935112 
loss at epoch : 1 and iter : 282 is : 0.03373432159423828 
loss at epoch : 1 and iter : 283 is : 0.08584365993738174 
loss at epoch : 1 and iter : 284 is : 0.09352383017539978 
loss at epoch : 1 and iter : 285 is : 0.05956628546118736 
loss at epoch : 1 and iter : 286 is : 0.02722092531621456 
loss at epoch : 1 and iter : 287 is : 0.04681972414255142 
loss at epoch : 1 and iter : 288 is : 0.06504137814044952 
loss at epoch : 1 and iter : 289 is : 0.04838448762893677 
loss at epoch : 1 and iter : 290 is : 0.038323137909173965 
loss at epoch : 1 and iter : 291 is : 0.1281844675540924 
loss at epoch : 1 and iter : 292 is : 0.12655729055404663 
loss at epoch : 1 and iter : 293 is : 0.03555428236722946 
loss at epoch : 1 and iter : 294 is : 0.07918030023574829 
loss at epoch : 1 and iter : 295 is : 0.0439404658973217 
loss at epoch : 1 and iter : 296 is : 0.04437912628054619 
loss at epoch : 1 and iter : 297 is : 0.017712270841002464

loss at epoch : 2 and iter : 75 is : 0.09273435175418854 
loss at epoch : 2 and iter : 76 is : 0.07806123793125153 
loss at epoch : 2 and iter : 77 is : 0.10796864330768585 
loss at epoch : 2 and iter : 78 is : 0.03830137103796005 
loss at epoch : 2 and iter : 79 is : 0.08205249160528183 
loss at epoch : 2 and iter : 80 is : 0.05141890048980713 
loss at epoch : 2 and iter : 81 is : 0.05048023536801338 
loss at epoch : 2 and iter : 82 is : 0.03666307404637337 
loss at epoch : 2 and iter : 83 is : 0.08905775099992752 
loss at epoch : 2 and iter : 84 is : 0.05084735155105591 
loss at epoch : 2 and iter : 85 is : 0.06945694983005524 
loss at epoch : 2 and iter : 86 is : 0.058265842497348785 
loss at epoch : 2 and iter : 87 is : 0.08785081654787064 
loss at epoch : 2 and iter : 88 is : 0.11444246768951416 
loss at epoch : 2 and iter : 89 is : 0.02016369253396988 
loss at epoch : 2 and iter : 90 is : 0.02069096639752388 
loss at epoch : 2 and iter : 91 is : 0.11716301739215851 
loss at epoch

loss at epoch : 2 and iter : 215 is : 0.10941672325134277 
loss at epoch : 2 and iter : 216 is : 0.07367604225873947 
loss at epoch : 2 and iter : 217 is : 0.11183430254459381 
loss at epoch : 2 and iter : 218 is : 0.058390434831380844 
loss at epoch : 2 and iter : 219 is : 0.0871834009885788 
loss at epoch : 2 and iter : 220 is : 0.14635708928108215 
loss at epoch : 2 and iter : 221 is : 0.08535098284482956 
loss at epoch : 2 and iter : 222 is : 0.04419022426009178 
loss at epoch : 2 and iter : 223 is : 0.05085649713873863 
loss at epoch : 2 and iter : 224 is : 0.1197875589132309 
loss at epoch : 2 and iter : 225 is : 0.11689697951078415 
loss at epoch : 2 and iter : 226 is : 0.10450331121683121 
loss at epoch : 2 and iter : 227 is : 0.14367422461509705 
loss at epoch : 2 and iter : 228 is : 0.11759328842163086 
loss at epoch : 2 and iter : 229 is : 0.05971644073724747 
loss at epoch : 2 and iter : 230 is : 0.045433301478624344 
loss at epoch : 2 and iter : 231 is : 0.1335998177528381

In [23]:
print(eval(net,train_loader,device=device))
#print(eval(net,validation_loader,device=device))

98.40495232931183


In [24]:
test_data = DancePredictionDataset("./dataset/test/",train=False,transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,shuffle=False)

In [25]:
final=[]
for images,names in test_loader:
    images=images.to(device)
    net.eval()
        
    out=net(images)
    v=out.argmax(-1)
    for i in range(len(names)):
        result=mapping[v[i].item()+1]
        name=names[i]
        final.append([name,result])

In [26]:
df_1=pd.DataFrame(final,columns=["Image","target"])
df_1.to_csv("output_3.csv",index=False)